# Dependancies

## Requirements

In [1]:
#!pip install sentence_transformers langchain openai tqdm datasets asyncio scikit-learn cohere tiktoken umap altair

In [2]:
import numpy as np
import re
import pandas as pd
from tqdm.notebook import tqdm
from datasets import load_dataset
import umap
import altair as alt
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer
from typing import List
import enum

from langchain_community.llms import Ollama
from langchain.output_parsers.regex_dict import RegexDictParser
from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage, SystemMessage, ChatMessage
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from pydantic import BaseModel, Field, validator, create_model
from openai import AsyncOpenAI, OpenAI
#import asyncio
import os

import requests

from pydantic import BaseModel, ValidationInfo, model_validator


import json

import itertools
from copy import deepcopy
from tqdm.notebook import tqdm, trange
from sklearn.cluster import KMeans

import umap.umap_ as umap
#import umap
import hdbscan

In [3]:
from src.bubble import *
from src.models import *
from src.utilities import *

Retrieved company Darty : 1707313014508x102198350946437700
Retrieved project Darty_trustpilot : 1707329196900x870734705097005300


In [4]:
prompts_path = "Prompts/fr/"

## Bubble API

In [5]:
feedbacks_df = get("Feedback", max_objects=100)

In [6]:
types_df = get("Type", constraints=[])
categories_df = get("Category")
original_subcategories_df = get("SubCategory")


In [7]:
company_infos = bubble_client.get(
    "Company",
    bubble_id=COMPANY_ID,
)
project_infos = bubble_client.get(
    "Project",
    bubble_id=PROJECT_ID,
)

In [8]:
types_df

,Modified Date,Created Date,Created By,Name,Description,_id
0,2024-02-07 13:29:49.626000+00:00,2024-02-07 13:27:22.244000+00:00,admin_user_feedback-analysis_test,Point positif,Élément apprécié par le client ou l'utilisateur,1707312442244x926718270622733700
1,2024-02-07 13:29:36.367000+00:00,2024-02-07 13:27:32.845000+00:00,admin_user_feedback-analysis_test,Nouvelle fonctionnalité,Suggestion d'évolution faite par le client ou ...,1707312452845x942796991724685900
2,2024-02-07 13:29:20.520000+00:00,2024-02-07 13:27:41.737000+00:00,admin_user_feedback-analysis_test,Point de douleur,Problème qui gène ou ennuie le client ou l'uti...,1707312461737x150602429464921100
3,2024-02-07 13:30:08.388000+00:00,2024-02-07 13:30:08.385000+00:00,admin_user_feedback-analysis_test,Bug,Anomalie de fonctionnement de l'application dé...,1707312608385x190928778572006180


In [9]:
TypeInsight = enum.Enum("Type de l'insight", [(convert_text_to_constants(x), x) for x in types_df.Name])
TypeInsight("Point positif")

<Type de l'insight.POINT_POSITIF: 'Point positif'>

In [10]:
TypeInsight.POINT_POSITIF

<Type de l'insight.POINT_POSITIF: 'Point positif'>

In [11]:
TypeInsight = enum.Enum("Type de l'insight", [(convert_text_to_constants(x), x) for x in types_df.Name])
CategoryInsight = enum.Enum("Categories de l'insight", [(convert_text_to_constants(x), x) for x in categories_df.Name])
dict_SubCategoriesInsight = {
    row["Name"]:enum.Enum("Categories de l'insight", [(convert_text_to_constants(x), x) for x in original_subcategories_df[original_subcategories_df["Parent_category"] == row["_id"]].Name])
    for _,row in categories_df.iterrows()
}

In [12]:
categories_df

,Created Date,Created By,Modified Date,Company,Name,Project,_id
0,2024-02-13 20:33:00.539000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:00.540000+00:00,1707313014508x102198350946437700,Satisfaction globale,1707329196900x870734705097005300,1707856380539x358769023739109480
1,2024-02-13 20:33:01.839000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:01.839000+00:00,1707313014508x102198350946437700,Expérience en magasin,1707329196900x870734705097005300,1707856381839x698419263697711700
2,2024-02-13 20:33:04.098000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:04.098000+00:00,1707313014508x102198350946437700,Qualité des produits,1707329196900x870734705097005300,1707856384098x905932661156251900
3,2024-02-13 20:33:05.648000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:05.648000+00:00,1707313014508x102198350946437700,Prix,1707329196900x870734705097005300,1707856385647x626212306295077200
4,2024-02-13 20:33:07.298000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:07.298000+00:00,1707313014508x102198350946437700,Service client,1707329196900x870734705097005300,1707856387298x901531512097841800
5,2024-02-13 20:33:09.036000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:09.036000+00:00,1707313014508x102198350946437700,Site web et application mobile,1707329196900x870734705097005300,1707856389036x464933483124026400
6,2024-02-13 20:33:10.966000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:10.966000+00:00,1707313014508x102198350946437700,Livraison à domicile,1707329196900x870734705097005300,1707856390966x807824640383989800
7,2024-02-13 20:33:12.725000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:12.725000+00:00,1707313014508x102198350946437700,Programme de fidélisation,1707329196900x870734705097005300,1707856392725x561938869661909600
8,2024-02-13 20:33:14.571000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:14.571000+00:00,1707313014508x102198350946437700,Recommandation,1707329196900x870734705097005300,1707856394571x761270792261882900
9,2024-02-13 20:33:15.250000+00:00,admin_user_feedback-analysis_test,2024-02-13 20:33:15.250000+00:00,1707313014508x102198350946437700,Suggestions d'amélioration,1707329196900x870734705097005300,1707856395250x201556099604734080


In [13]:
df  = categories_df.copy()
df['Parent_category'] = df['_id'].astype(str)
original_subcategories_df['Parent_category'] = original_subcategories_df['Parent_category'].astype(str)
df = pd.merge(original_subcategories_df, df, on=["Parent_category", "Company"])
df = df[["Name_x", "Name_y", "Company", "Description", "_id_x", "_id_y"]]
df.columns = ["Name", "Category",  "Company", "Description", "_id", "Category_id"]
subcategories_df = df
subcategories_df.head()

,Name,Category,Company,Description,_id,Category_id
0,Note globale,Satisfaction globale,1707313014508x102198350946437700,Indique le niveau de satisfaction globale du c...,1707856380878x190932766251103330,1707856380539x358769023739109480
1,Commentaire général,Satisfaction globale,1707313014508x102198350946437700,Permet au client de s'exprimer librement sur s...,1707856381262x125036210254948700,1707856380539x358769023739109480
2,Propreté du magasin,Expérience en magasin,1707313014508x102198350946437700,Évalue la propreté du magasin et des rayons.,1707856382237x200998574626378050,1707856381839x698419263697711700
3,Achalandage,Expérience en magasin,1707313014508x102198350946437700,Indique si le magasin était trop achalandé ou ...,1707856382601x227919930302718180,1707856381839x698419263697711700
4,Amabilité du personnel,Expérience en magasin,1707313014508x102198350946437700,Évalue la gentillesse et la serviabilité du pe...,1707856382994x231229813327290530,1707856381839x698419263697711700


In [14]:
SubCategoriyInsight = enum.Enum("Categories de l'insight", [(row["Category"]+" : "+row["Name"], convert_text_to_constants(row["Category"]+" : "+row["Name"])) for _, row in subcategories_df.iterrows()])

In [15]:
types_descr = columns_to_string(types_df, "Name", "Description")
print(types_descr)


Point positif : Élément apprécié par le client ou l'utilisateur
Nouvelle fonctionnalité : Suggestion d'évolution faite par le client ou l'utilisateur
Point de douleur : Problème qui gène ou ennuie le client ou l'utilisateur
Bug : Anomalie de fonctionnement de l'application détectée par l'utilisateur


In [16]:
tags_descr = columns_to_string(subcategories_df, "Name", "Description")
print(tags_descr)

Note globale : Indique le niveau de satisfaction globale du client vis-à-vis de Darty.
Commentaire général : Permet au client de s'exprimer librement sur son expérience chez Darty.
Propreté du magasin : Évalue la propreté du magasin et des rayons.
Achalandage : Indique si le magasin était trop achalandé ou si les files d'attente étaient trop longues.
Amabilité du personnel : Évalue la gentillesse et la serviabilité du personnel.
Disponibilité des produits : Indique si les produits recherchés par le client étaient disponibles en magasin.
Délais d'attente en caisse : Évalue le temps d'attente en caisse.
Fraîcheur des produits : Évalue la fraîcheur des produits frais et périssables.
Variété des produits : Indique si le choix de produits est suffisant et répond aux attentes du client.
Qualité des produits MDD : Évalue la qualité des produits de la marque Darty.
Niveau des prix : Indique si le client trouve les prix pratiqués par Darty trop élevés.
Promotions et offres spéciales : Évalue la

In [17]:
#example_insight = "Manque de clarté de l'affichage des prix en magasin"
#exemple_commentaire = "je suis exclusif metro je n ai aucun representant j achetais jusqu a present tout metro par facilite mais je suis tres souvent décue par la reponse ha non on n en a pas cela arrive demain je pense que depuis le covid tout le monde ou presque s en fou!!!"
#examples_insights_df = pd.DataFrame([
#    {"Insights qui devraient en découler": "Déceptions face aux retards de livraison"},
#    {"Insights qui devraient en découler": "Impression d'une baisse de qualité du service depuis le Covid"},
#])

feedback_context = {
            "entreprise": company_infos["Name"],
            "context": company_infos['Context'],
            "role": company_infos['Role'],
            "cible": project_infos['Target'],
            "insight_types": types_descr,
            "insight_categories": tags_descr,
            #"question": project_infos['Study_question'],
            #"exemple_commentaire": exemple_commentaire,
            #"example_insights": '\n- '.join(list(examples_insights_df['Insights qui devraient en découler'])),
        }

feedback_context

{'entreprise': 'Darty',
 'context': 'Fondée en 1957, Darty est une enseigne française spécialisée dans la distribution d\'électroménager, d\'équipements électroniques et de produits culturels. Rachetée par la Fnac en 2016, elle est aujourd\'hui l\'un des leaders européens de la distribution omnicanale.\n\nÉvènements récents:\n\n    2016: Rachat par la Fnac et création du groupe Fnac Darty.\n    2017: Lancement de la marketplace Darty.com.\n    2018: Déploiement du "Contrat de Confiance Fnac Darty" dans tous les magasins.\n    2019: Lancement de l\'offre de services "Darty+."\n    2020: Accélération de la transformation digitale du groupe.\n    2021: Acquisition de Mistergooddeal, spécialiste du e-commerce en produits reconditionnés.\n    2022: Lancement de la Fnac Darty Academy, une plateforme de formation en ligne.\n\nConcurrents:\n\n    Boulanger\n    Conforama\n    Gitem\n    Amazon\n    Cdiscount\n\nEnjeux:\n\n    Darty doit faire face à une concurrence accrue sur le marché de l\'é

# Insights extraction

### Aspects and Insights creation

In [18]:
#FeedbackIndex = enum.Enum("Indice du retour associé", [(str(i), i) for i in range(BATCH_SIZE)])

class SousCategorie(BaseModel):
    indice: int = Field(description="Indice de la sous-catégorie. Doit être un entier.")
    nom: str = Field(description="Nom de la sous-catégorie.")

    def __str__(self):
        return self.nom + ' ('+str(self.indice)+')'

    @model_validator(mode="after")
    def validate_ids(self, info: ValidationInfo):
        context = info.context
        if context:
            tags: List[SousCategorie] = context.get("sous_categories")
            assert self.indice in {
                tag.indice for tag in tags
            }, f"sous_categories ID {self.indice} not found in context"
            assert self.nom in {
                tag.nom for tag in tags
            }, f"sous_categories name {self.nom} not found in context"
        return self
    
class SousCategorieAvecDescription(SousCategorie):
    categorie : str
    description: str


class Aspect(BaseModel):
    sous_categorie : SousCategorie = Field(description="Sous-catégorie concernée.")
    note_satisfaction : int = Field(description="Note de satisfaction du client concernant cette sous-catégorie, de 1 (pas content) à 5 (très content).")
    explication: str = Field(description="Eventuelle explication du ressenti du client, si celle-ci parait importante à faire remonter au sein de l'entreprise. Doit être aussi claire et concise que possible.") #Field(description="Point intéressant a retenir du commentaire.")


    def __str__(self):
        return '\n' + str(self.sous_categorie) + '\nSatisfaction: ' + str(self.note_satisfaction) + "/5\nExplication: " + self.explication
    
    @model_validator(mode="after")
    def validate_ids(self, info: ValidationInfo):
        assert (0 <= self.note_satisfaction) and (self.note_satisfaction <= 5)
        return self

class ListAspects(BaseModel):
    list_aspects: List[Aspect] = Field(description="Liste des différents aspects évoqués dans le feedback.")

    def __str__(self):
        return '\n'.join([str(x) for x in self.list_aspects])
    

class AspectsRequest(BaseModel):
    texts: List[str]
    sous_categories: List[SousCategorieAvecDescription]


class AspectsResponse(BaseModel):
    texts: List[str]
    predictions: List[Aspect]

In [19]:
sous_categories = [SousCategorieAvecDescription(indice=i, nom=row["Category"] + " : " + row["Name"], categorie=row["Category"], description=row["Description"]) for i ,row in  subcategories_df.iterrows()]
sous_categories

[SousCategorieAvecDescription(indice=0, nom='Satisfaction globale : Note globale', categorie='Satisfaction globale', description='Indique le niveau de satisfaction globale du client vis-à-vis de Darty.'),
 SousCategorieAvecDescription(indice=1, nom='Satisfaction globale : Commentaire général', categorie='Satisfaction globale', description="Permet au client de s'exprimer librement sur son expérience chez Darty."),
 SousCategorieAvecDescription(indice=2, nom='Expérience en magasin : Propreté du magasin', categorie='Expérience en magasin', description='Évalue la propreté du magasin et des rayons.'),
 SousCategorieAvecDescription(indice=3, nom='Expérience en magasin : Achalandage', categorie='Expérience en magasin', description="Indique si le magasin était trop achalandé ou si les files d'attente étaient trop longues."),
 SousCategorieAvecDescription(indice=4, nom='Expérience en magasin : Amabilité du personnel', categorie='Expérience en magasin', description='Évalue la gentillesse et la s

In [20]:
with open(prompts_path+'prompt_aspects.txt') as f:
    prompt_aspects = PromptTemplate.from_template(f.read())

print(prompt_aspects.template)

Analyse le commentaire client qui te sera donné en effectuant les étapes suivantes:

1) Détermine toutes les sous-catégories concernées par le commentaire, et uniquement les sous-catégories concernées par le commentaire. Pour chacune d'entre elle, renvoit son nom et son indice, qui est un entier.

2) Pour chaque sous-catégorie, indique le niveau de satisfaction du client relativement à cette sous-catégorie avec une note de 1 à 5. 

3) Si le client a donné une explication à son niveau de satisfaction, et que celle ci te sembel intéressante a faire remonter au sein de ton entreprise, ajoute la. Si il n'y a pas d'explication, ou que celle ci n'est pas très intéressante, renvoit une string vide. Une explication intéressante doit permettre d'améliorer le produit, le servide rendu au client ou d'ajuster la stratégie de l'entreprise.  

Par exemple, si pour l'entreprise Amazon on analysait le commentaire suivant:
"le produit est super cool, mais la livraison était catastrophique… Ils ont lais

In [21]:
", ".join([f"`{tag}`" for tag in sous_categories])

"`Satisfaction globale : Note globale (0)`, `Satisfaction globale : Commentaire général (1)`, `Expérience en magasin : Propreté du magasin (2)`, `Expérience en magasin : Achalandage (3)`, `Expérience en magasin : Amabilité du personnel (4)`, `Expérience en magasin : Disponibilité des produits (5)`, `Expérience en magasin : Délais d'attente en caisse (6)`, `Qualité des produits : Fraîcheur des produits (7)`, `Qualité des produits : Variété des produits (8)`, `Qualité des produits : Qualité des produits MDD (9)`, `Prix : Niveau des prix (10)`, `Prix : Promotions et offres spéciales (11)`, `Service client : Efficacité du service client (12)`, `Service client : Amabilité du service client (13)`, `Site web et application mobile : Facilité d'utilisation (14)`, `Site web et application mobile : Fonctionnalités (15)`, `Site web et application mobile : Rapidité de livraison (16)`, `Livraison à domicile : Délai de livraison (17)`, `Livraison à domicile : État des produits livrés (18)`, `Livraiso

In [22]:
context

{'company': 'Darty', 'project': 'Darty_trustpilot', 'context': None}

In [23]:
feedback_context

{'entreprise': 'Darty',
 'context': 'Fondée en 1957, Darty est une enseigne française spécialisée dans la distribution d\'électroménager, d\'équipements électroniques et de produits culturels. Rachetée par la Fnac en 2016, elle est aujourd\'hui l\'un des leaders européens de la distribution omnicanale.\n\nÉvènements récents:\n\n    2016: Rachat par la Fnac et création du groupe Fnac Darty.\n    2017: Lancement de la marketplace Darty.com.\n    2018: Déploiement du "Contrat de Confiance Fnac Darty" dans tous les magasins.\n    2019: Lancement de l\'offre de services "Darty+."\n    2020: Accélération de la transformation digitale du groupe.\n    2021: Acquisition de Mistergooddeal, spécialiste du e-commerce en produits reconditionnés.\n    2022: Lancement de la Fnac Darty Academy, une plateforme de formation en ligne.\n\nConcurrents:\n\n    Boulanger\n    Conforama\n    Gitem\n    Amazon\n    Cdiscount\n\nEnjeux:\n\n    Darty doit faire face à une concurrence accrue sur le marché de l\'é

In [24]:
def tag_single_request(prompt: str, sous_categories: List[SousCategorie]) -> Aspect:
    allowed_tags = [(tag.indice, tag.nom) for tag in sous_categories]
    allowed_tags_str = ", ".join([f"`{tag}`" for tag in allowed_tags])

    return client.chat.completions.create(
        model="mixtral",
        messages=[
            {
                "role": "system",
                "content": f"Tu es {feedback_context['role']} au sein de l'entreprise {feedback_context['entreprise']}. Voici un bref rappel sur cette entreprise: \n'{feedback_context['context']}'\n\En tant que  {feedback_context['role']}, tu est spécialisé dans l'analyse de commentaire."
            },
            {"role": "user", "content": prompt},
            {
                "role": "user",
                "content": f"Voici les sous-catégories: {allowed_tags_str}",
            },
        ], 
        response_model=ListAspects,  # Minimizes the hallucination of tags that are not in the allowed tags.
        validation_context={"sous_categories": sous_categories},
    )

def tag_request(request: AspectsRequest) -> AspectsResponse:
    predictions = [tag_single_request(text, request.tags) for text in request.texts]
    
    return AspectsResponse(
        texts=request.texts,
        predictions=predictions)

In [25]:
#feedback ="J'ai commandé une paire de chaussures sur votre site. Le site était facile à utiliser mais j'ai galéré à trouver ma taille. La livraison a été super rapide, mais les chaussures étaient trop petites. J'ai contacté le service client pour les renvoyer et ils m'ont dit que je devais payer les frais de retour. Du coup, j'ai décidé de les garder et de les donner à ma sœur. Elles sont bien mais un peu trop serrées pour moi."
feedback ="I ordered a pair of shoes on your site. The site was easy to use but I had a hard time finding my size. The delivery was super fast, but the shoes were too small. I contacted the customer service to return them and they told me I had to pay the return shipping. So I decided to keep them and give them to my sister. They are good but a little too tight for me."
prompt = prompt_aspects.invoke({"feedback": feedback})
response = tag_single_request(prompt.text, sous_categories=sous_categories)
print(response)


Site web et application mobile : Facilité d'utilisation (14)
Satisfaction: 5/5
Explication: 

Site web et application mobile : Rapidité de livraison (16)
Satisfaction: 5/5
Explication: 

Qualité des produits : Fraîcheur des produits (7)
Satisfaction: 3/5
Explication: La taille des chaussures commandées ne correspondait pas à la taille réelle du produit.

Programme de fidélisation : Avantages du programme (20)
Satisfaction: 1/5
Explication: Le client a dû payer les frais de retour, ce qui a affecté sa satisfaction globale.


In [27]:
for i, feedback in tqdm(feedbacks_df.iterrows()):
    prompt = prompt_aspects.invoke({"feedback": feedback['Content']})
    print("prompt", prompt.text)
    response = tag_single_request(prompt.text, sous_categories=sous_categories)
    print("response", response)
    results = bubble_client.create(
        "Aspect",
        [{
            "Company": COMPANY_ID,
            "Project": PROJECT_ID,
            "Category": subcategories_df.loc[aspect.sous_categorie.indice, "Category_id"],
            "Consequence": "",
            "Explanation": aspect.explication,
            "Rating": aspect.note_satisfaction,
            "Sub_category": subcategories_df.loc[aspect.sous_categorie.indice, "_id"],
            "Associated_feedback": feedback["_id"],
            }  for aspect in response.list_aspects]
        )

    bubble_client.update_object(bubble_type="Feedback", bubble_id=feedback["_id"], fields={"Aspects": [res['id'] for res in results]})


0it [00:00, ?it/s]

KeyboardInterrupt: 

In [ ]:
assert False

In [ ]:
with open(prompts_path+'prompt_insights_creation.txt') as f:
    prompt_insights = PromptTemplate.from_template(f.read())

In [ ]:
prompts = []
BATCH_SIZE = 10

for batch_df in batchify(feedbacks_df, size=BATCH_SIZE):
    context = deepcopy(feedback_context)
    context["feedbacks"] = '\n\n'.join([str(i)+" : "+x for i, x in zip(batch_df.index, batch_df["Content"])])  
    #"- "+"\n- ".join(batch_df['content'])
    #context["insights"] = "- "+"\n- ".join(batch_df['content'])
    prompts.append(prompt_insights.invoke(context))

print(len(prompts))

10


In [ ]:
print(prompts[0].text)

Tu es analyste marketing au sein de l'entreprise suivante:
'''
Fondée en 1957, Darty est une enseigne française spécialisée dans la distribution d'électroménager, d'équipements électroniques et de produits culturels. Rachetée par la Fnac en 2016, elle est aujourd'hui l'un des leaders européens de la distribution omnicanale.

Évènements récents:

    2016: Rachat par la Fnac et création du groupe Fnac Darty.
    2017: Lancement de la marketplace Darty.com.
    2018: Déploiement du "Contrat de Confiance Fnac Darty" dans tous les magasins.
    2019: Lancement de l'offre de services "Darty+."
    2020: Accélération de la transformation digitale du groupe.
    2021: Acquisition de Mistergooddeal, spécialiste du e-commerce en produits reconditionnés.
    2022: Lancement de la Fnac Darty Academy, une plateforme de formation en ligne.

Concurrents:

    Boulanger
    Conforama
    Gitem
    Amazon
    Cdiscount

Enjeux:

    Darty doit faire face à une concurrence accrue sur le marché de l'éle

In [ ]:
responses = apply_analysis(prompts, InsightsList, bar=True)
list_batch_insights_df = [pd.DataFrame(enum_to_str(response.insights_list)) for response in responses]

print(len(list_batch_insights_df), "batch have been processed")

0it [00:00, ?it/s]

10 batch have been processed


In [ ]:
responses[0].insights_list


[FirstInsight(associated_indexes=[0], contenu='Les clients apprécient le service de récupération rapide des produits en magasin et trouvent le personnel accueillant et sympathique.')]

In [ ]:
[len(df) for df in list_batch_insights_df]

[10, 5]

In [ ]:
list_batch_insights_df[0]

,insight_categories,insight_type,associated_feedbacks,contenu
0,[Livraison],Point de douleur,[None],La limitation de certains produits à l'achat e...
1,"[Expérience d'achat, Politique de prix]",Point de douleur,[None],L'offre en ligne est perçue comme moins avanta...
2,[Magasin],Point de douleur,[None],"Le rangement en magasin est jugé désordonné, r..."
3,[],Point positif,[None],Les opérations de déstockage chez d'autres fou...
4,[Expérience d'achat],Point de douleur,[None],L'arrêt de la distribution des promotions sous...
5,"[Qualité des produits, Politique de prix]",Point de douleur,[None],Les prix de certains produits chez Metro sont ...
6,[Qualité des produits],Point de douleur,[None],La disparition de l'offre d'entrées chaudes et...
7,[Service client],Point de douleur,[None],Un manque de support pour charger les achats e...
8,[Politique de prix],Point de douleur,[None],"Les prix sont jugés trop élevés, même avec un ..."
9,"[Livraison, Service client]",Point de douleur,[None],Le processus de livraison et la gestion des av...


In [ ]:
list(list_batch_insights_df[0]['contenu'])

["La limitation de certains produits à l'achat en carton lors de la livraison contrairement à l'achat à l'unité en magasin réduit la flexibilité pour les clients.",
 "L'offre en ligne est perçue comme moins avantageuse que l'achat en magasin, avec une impossibilité de comparer les prix et un sentiment d'obligation de se rendre en magasin.",
 'Le rangement en magasin est jugé désordonné, rendant difficile la recherche de produits.',
 "Les opérations de déstockage chez d'autres fournisseurs sont très appréciées.",
 "L'arrêt de la distribution des promotions sous forme papier ou digitale est regretté.",
 'Les prix de certains produits chez Metro sont jugés plus élevés que ceux de la grande distribution, même pour des produits identiques.',
 "La disparition de l'offre d'entrées chaudes et de propositions traiteurs pendant les fêtes de fin d'année est regrettée.",
 'Un manque de support pour charger les achats en grande quantité dans le véhicule du client est signalé.',
 'Les prix sont jugé

## Accociate newly created insights to feedbacks 

In [ ]:
with open(prompts_path+'prompt_feedbacks.txt') as f:
    prompt_feedbacks = PromptTemplate.from_template(f.read())

In [ ]:
class Sentiment(str, enum.Enum):
    POSITIF = "Positif"
    NEUTRE = "Neutre"
    NEGATIF = "Négatif"


In [ ]:
InsightsIndex = enum.Enum("Indice de l'insight associé", [(str(i), i) for i in range(BATCH_SIZE)])

class Feedback(BaseModel):
        insights_list: List[InsightsIndex] = Field(description="Indices des insights associés à ce retour")
        sentiment: Sentiment = Field(description="Sentiment exprimé")

class FeedbackInfosList(BaseModel):
        feedbacks_list: List[Feedback] = Field(description="Liste des informations associées aux feedbacks.")

In [ ]:
prompts = []
for batch_insights_df, batch_feedbacks_df in zip(list_batch_insights_df, batchify(feedbacks_df, size=BATCH_SIZE)):
    #InsightsEnum = enum.Enum("Insight associé", [(convert_text_to_constants(x), i) for i, x in zip(batch_insights_df.index, batch_insights_df["content"])])

    context = deepcopy(feedback_context)
    #context["feedbacks"] = "- "+"\n- ".join(batch_feedbacks_df['content'])
    context["feedbacks"] = '\n'.join([str(i)+" : "+x for i, x in zip(batch_insights_df.index, batch_feedbacks_df["content"])])  
    context["insights"] = '\n'.join([str(i)+" : "+x for i, x in zip(batch_insights_df.index, batch_insights_df["contenu"])])
    prompts.append(prompt_feedbacks.invoke(context))



In [ ]:
print(prompts[0].text)

Tu es analyste marketing au sein de l'entreprise suivante:
METRO est une enseigne réservée aux professionnels des métiers de bouche (restaurateurs, traiteurs, bouchers, boulangers…) qui vend des produits alimentaires et non alimentaires adaptés à leur activité, en magasins (appelés entrepôts ou halles) et en livraison. 

Tu as mené une enquête auprès des client de l'entreprise. 
Tu as récupérés des commentaires, et en a extrait des insights.
               
Pour chacun des retours qui te seront donnés, effectue les étapes suivantes:

Étape 1 - Identifie si le sentiment exprimé dans chacun des retours par le client est "Positif", "Neutre" ou "Négatif". Prends en compte la formulation de la question ayant été posée (Il s’agit d’une étude clients pour mesurer l’image prix de l’enseigne METRO. En fin de questionnaire, une question ouverte permet aux clients de laisser s’ils le souhaitent un commentaire libre. Ils peuvent exprimer leurs points de satisfaction et d’insatisfaction sur les pri

In [ ]:

responses = apply_async_analysis(prompts, FeedbackInfosList)

list_enriched_feedbacks_df = [pd.DataFrame(enum_to_str(response.feedbacks_list)) for response in responses]

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_68618/2334417953.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  return enum_to_str(e.dict())


In [ ]:
[len(df) for df in list_enriched_feedbacks_df]

[9, 5]

In [ ]:
len(pd.concat(list_enriched_feedbacks_df))

14

In [ ]:
for batch_insights_df, batch_index_feedbacks, enriched_feedbacks_df in zip(list_batch_insights_df, batchify(feedbacks_df.index, size=BATCH_SIZE), list_enriched_feedbacks_df):
    feedbacks_df.loc[batch_index_feedbacks, 'sentiment'] = enriched_feedbacks_df['sentiment']
    feedbacks_df.loc[batch_index_feedbacks, 'insights_index'] = enriched_feedbacks_df['insights_list']

In [ ]:
batch_insights_df

,insight_categories,insight_type,contenu
0,[Politique de prix],Point de douleur,"Les prix de certains produits (bananes, frambo..."
1,[Qualité des produits],Point de douleur,"La qualité et la fraîcheur des produits, notam..."
2,[Expérience d'achat],Point de douleur,Le nouveau directeur de Metro Caen est perçu c...
3,"[Disponibilité des produits, Livraison]",Point de douleur,La réduction de l'assortiment disponible à la ...
4,[Politique de prix],Point de douleur,Les promotions sont jugées insuffisantes pour ...
5,[],Nouvelle demande,Mise en place de points de fidélité suggérée.


In [ ]:
list_batch_feedbacks_df = [pd.DataFrame(enum_to_str(response.feedbacks_list)) for response in responses]
list_batch_feedbacks_df

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_68618/2334417953.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  return enum_to_str(e.dict())


[  insights_list sentiment
 0            []   Positif
 1           [0]   Négatif
 2           [1]   Négatif
 3           [2]   Positif
 4           [3]   Négatif
 5           [4]   Négatif
 6           [5]   Négatif
 7           [6]   Négatif
 8           [7]   Négatif,
   insights_list sentiment
 0           [4]   Négatif
 1           [5]   Positif
 2           [3]   Négatif
 3        [0, 1]   Négatif
 4           [2]   Négatif]

In [ ]:
feedbacks_df

,Modified Date,Created Date,Created By,content,company,sentiment,Analyzed?,source,character_number,insights,_id,insights_index
0,2024-01-31 15:19:21.203000+00:00,2024-01-21 15:40:00.025000+00:00,1705847494855x437900943146650500,livrer TOUT les produits disponibles en magasi...,1705585399217x205117684451615600,Positif,False,1705851599107x404539534708310000,95,"[1706714349371x275598239568271680, 17067143467...",1705851599759x115801332943705310,[]
1,2024-01-31 15:19:21.577000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,Votre offer internet devient « ridicule ». Ell...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,486,"[1706714348320x182894283191188160, 17067143465...",1705851599759x118530353766926000,[0]
2,2024-01-31 15:19:21.952000+00:00,2024-01-21 15:39:59.898000+00:00,1705847494855x437900943146650500,Le rangement est bordélique une vache ne retro...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,66,"[1706714348689x648871534716904700, 17067143464...",1705851599759x119429130200745520,[1]
3,2024-01-25 13:15:45.483000+00:00,2024-01-21 15:40:00.012000+00:00,1705847494855x437900943146650500,"Je profite ailleurs d'opération de destockage,...",1705585399217x205117684451615600,Positif,False,1705851599107x404539534708310000,63,[],1705851599759x120869695273613470,[2]
4,2024-01-31 15:19:22.739000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,il est dommage de ne plus recevoir les promoti...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,78,"[1706714349210x970420167460772200, 17067143468...",1705851599759x123910318505263460,[3]
5,2024-01-31 15:19:23.357000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,Je ne comprends pas comment un grossiste comme...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,240,"[1706714348317x477146000760127360, 17067143465...",1705851599759x133516960359271180,[4]
6,2024-01-31 15:19:23.807000+00:00,2024-01-21 15:40:00.210000+00:00,1705847494855x437900943146650500,Nous regrettons que Métro ne propose plus d'en...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,237,"[1706714349391x414238817914070600, 17067143467...",1705851599759x139047234470437000,[5]
7,2024-01-31 15:19:24.126000+00:00,2024-01-21 15:40:00.024000+00:00,1705847494855x437900943146650500,metro devrai mieux s'occuper des client sutout...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,179,"[1706714349750x607338702255573100, 17067143464...",1705851599759x147056969853707870,[6]
8,2024-01-31 15:19:24.532000+00:00,2024-01-21 15:40:00.025000+00:00,1705847494855x437900943146650500,Les prix sont trop élevés même avec le système...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,221,"[1706714348627x226373109662698180, 17067143465...",1705851599759x147526339935997200,[7]
9,2024-01-31 15:19:24.997000+00:00,2024-01-21 15:39:59.898000+00:00,1705847494855x437900943146650500,livraison a revoir et les avoirs tpujours enat...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,51,"[1706714349730x279883762960940960, 17067143465...",1705851599759x160186801121862100,NaN


In [ ]:
list_batch_feedbacks_df[0]

,insights_list,sentiment
0,[],Positif
1,[0],Négatif
2,[1],Négatif
3,[2],Positif
4,[3],Négatif
5,[4],Négatif
6,[5],Négatif
7,[6],Négatif
8,[7],Négatif


In [ ]:
list_batch_insights_df[-1]

,insight_categories,insight_type,contenu
0,[Politique de prix],Point de douleur,"Les prix de certains produits (bananes, frambo..."
1,[Qualité des produits],Point de douleur,"La qualité et la fraîcheur des produits, notam..."
2,[Expérience d'achat],Point de douleur,Le nouveau directeur de Metro Caen est perçu c...
3,"[Disponibilité des produits, Livraison]",Point de douleur,La réduction de l'assortiment disponible à la ...
4,[Politique de prix],Point de douleur,Les promotions sont jugées insuffisantes pour ...
5,[],Nouvelle demande,Mise en place de points de fidélité suggérée.


In [ ]:
[x for x in batchify(feedbacks_df, size=BATCH_SIZE)][-1]

,Modified Date,Created Date,Created By,content,company,sentiment,Analyzed?,source,character_number,insights,_id,insights_index
10,2024-01-31 15:19:25.367000+00:00,2024-01-21 15:40:00.113000+00:00,1705847494855x437900943146650500,pas assez de promos pour petite quantité achet...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,67,"[1706714349262x615989589241161500, 17067143465...",1705851599759x161751993329532380,NaN
11,2024-01-31 15:19:25.696000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,mettre en place des points fidélités en,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,39,"[1706714348932x434155294366872260, 17067143464...",1705851599759x162340632957431520,NaN
12,2024-01-31 15:19:26.001000+00:00,2024-01-21 15:39:59.886000+00:00,1705847494855x437900943146650500,il serait souhaitable de remettre de l assorti...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,272,"[1706714347762x482499971709037250, 17067143469...",1705851599759x170076823432107940,NaN
13,2024-01-31 15:19:26.402000+00:00,2024-01-21 15:40:00.210000+00:00,1705847494855x437900943146650500,Produits pas très réguliers en qualité et en f...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,246,"[1706714348627x226373109662698180, 17067143465...",1705851599759x175471476046099360,NaN
14,2024-01-31 15:19:26.738000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,que le nouveau directeur de metro caen apprenn...,1705585399217x205117684451615600,NaN,False,1705851599107x404539534708310000,67,"[1706714348645x391695391069547500, 17067143465...",1705851599759x188863221495132220,NaN


In [ ]:
[len(df) for df in list_batch_feedbacks_df]

[9, 5]

In [ ]:

l = [response.feedbacks_list for response in responses]
l = list(itertools.chain.from_iterable(l))
feedbacks_infos_df = pd.DataFrame(enum_to_str(l))
feedbacks_infos_df

/var/folders/bs/0f5dcrc501sf9wtpqltg62940000gn/T/ipykernel_68618/2334417953.py:11: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.5/migration/
  return enum_to_str(e.dict())


,insights_list,sentiment
0,[],Positif
1,[0],Négatif
2,[1],Négatif
3,[2],Positif
4,[3],Négatif
5,[4],Négatif
6,[5],Négatif
7,[6],Négatif
8,[7],Négatif
9,[4],Négatif


In [ ]:
feedbacks_infos_df

,insights_list,sentiment
0,[],Positif
1,[0],Négatif
2,[1],Négatif
3,[2],Positif
4,[3],Négatif
5,[4],Négatif
6,[5],Négatif
7,[6],Négatif
8,[7],Négatif
9,[4],Négatif


In [ ]:
feedbacks_infos_df

,insights_list,sentiment
0,[],Positif
1,[0],Négatif
2,[1],Négatif
3,[2],Positif
4,[3],Négatif
5,[4],Négatif
6,[5],Négatif
7,[6],Négatif
8,[7],Négatif
9,[4],Négatif


In [ ]:
feedbacks_df['sentiment'] = feedbacks_infos_df['sentiment']
feedbacks_df['insights_list'] = feedbacks_infos_df['insights_list']
feedbacks_df

,Modified Date,Created Date,Created By,content,company,sentiment,Analyzed?,source,character_number,insights,_id,insights_index,insights_list
0,2024-01-31 15:19:21.203000+00:00,2024-01-21 15:40:00.025000+00:00,1705847494855x437900943146650500,livrer TOUT les produits disponibles en magasi...,1705585399217x205117684451615600,Positif,False,1705851599107x404539534708310000,95,"[1706714349371x275598239568271680, 17067143467...",1705851599759x115801332943705310,[],[]
1,2024-01-31 15:19:21.577000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,Votre offer internet devient « ridicule ». Ell...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,486,"[1706714348320x182894283191188160, 17067143465...",1705851599759x118530353766926000,[0],[0]
2,2024-01-31 15:19:21.952000+00:00,2024-01-21 15:39:59.898000+00:00,1705847494855x437900943146650500,Le rangement est bordélique une vache ne retro...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,66,"[1706714348689x648871534716904700, 17067143464...",1705851599759x119429130200745520,[1],[1]
3,2024-01-25 13:15:45.483000+00:00,2024-01-21 15:40:00.012000+00:00,1705847494855x437900943146650500,"Je profite ailleurs d'opération de destockage,...",1705585399217x205117684451615600,Positif,False,1705851599107x404539534708310000,63,[],1705851599759x120869695273613470,[2],[2]
4,2024-01-31 15:19:22.739000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,il est dommage de ne plus recevoir les promoti...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,78,"[1706714349210x970420167460772200, 17067143468...",1705851599759x123910318505263460,[3],[3]
5,2024-01-31 15:19:23.357000+00:00,2024-01-21 15:39:59.800000+00:00,1705847494855x437900943146650500,Je ne comprends pas comment un grossiste comme...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,240,"[1706714348317x477146000760127360, 17067143465...",1705851599759x133516960359271180,[4],[4]
6,2024-01-31 15:19:23.807000+00:00,2024-01-21 15:40:00.210000+00:00,1705847494855x437900943146650500,Nous regrettons que Métro ne propose plus d'en...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,237,"[1706714349391x414238817914070600, 17067143467...",1705851599759x139047234470437000,[5],[5]
7,2024-01-31 15:19:24.126000+00:00,2024-01-21 15:40:00.024000+00:00,1705847494855x437900943146650500,metro devrai mieux s'occuper des client sutout...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,179,"[1706714349750x607338702255573100, 17067143464...",1705851599759x147056969853707870,[6],[6]
8,2024-01-31 15:19:24.532000+00:00,2024-01-21 15:40:00.025000+00:00,1705847494855x437900943146650500,Les prix sont trop élevés même avec le système...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,221,"[1706714348627x226373109662698180, 17067143465...",1705851599759x147526339935997200,[7],[7]
9,2024-01-31 15:19:24.997000+00:00,2024-01-21 15:39:59.898000+00:00,1705847494855x437900943146650500,livraison a revoir et les avoirs tpujours enat...,1705585399217x205117684451615600,Négatif,False,1705851599107x404539534708310000,51,"[1706714349730x279883762960940960, 17067143465...",1705851599759x160186801121862100,NaN,[4]


## Feedbacks attribution

In [ ]:
insights_enum = enum.Enum("Insight associé", [(convert_text_to_constants(x), i) for i, x in zip(batch_insights_df.index, batch_insights_df["content"])])

In [ ]:
with open(prompts_path+'prompt_feedbacks.txt') as f:
    prompt_feedbacks = PromptTemplate.from_template(f.read())

input_variables=['cible', 'context', 'feedbacks', 'insights', 'question', 'role'] template='Tu es {role} au sein de l\'entreprise suivante:\n{context}\n\nTu as mené une enquête auprès des {cible} de l\'entreprise. \nTu as récupérés des commentaires, et en a extrait des insights.\n               \nPour chacun des retours qui te seront donnés, effectue les étapes suivantes:\n\nÉtape 1 - Identifie si le sentiment exprimé dans chacun des retours par le {cible} est "Positif", "Neutre" ou "Négatif". Prends en compte la formulation de la question ayant été posée ({question}) afin de bien interpréter le sens du retour {cible}.\nN\'oublie pas l\'accent si tu choisis Négatif, il n\'est pas correct d\'écrire Negatif.\n               \nÉtape 2 - Associe au retour les indices des éventuels insights qui y sont associés.\nUn insight doit nécessairement être associé à au moins un retour. Un retour peut n\'être associé à aucun insight.\n\nVoici les retours à traiter:\n\n\'\'\'\n{feedbacks}\n\'\'\'\n\nE

In [ ]:
feedback_parser = PydanticOutputParser(pydantic_object=Feedback)

prompt_feedback = PromptTemplate.from_template(
    template= prompt_template_feedback,
    partial_variables= {"format_instructions": feedback_parser.get_format_instructions()},
)

prompts = []
for feedback in feedbacks_df[feedbacks_column]:
    context = deepcopy(feedback_context)
    context["feedback"] = feedback
    prompts.append(prompt_feedback.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, feedback_parser)

feedbacks_df["sentiment"] = [rep.sentiment for rep in parsed_responses]
feedbacks_df["insights"] = [[] for rep in parsed_responses]

k=0
insights = []
for i, rep in enumerate(parsed_responses):
    for j, insight in enumerate(rep.insights_list):
        insights.append(insight)
        feedbacks_df["insights"].iloc[i].append(str(k))
        k += 1

In [ ]:
feedbacks_df.head()

In [ ]:
insights_df = pd.DataFrame({
    "content":insights,
    "feedback_count": 1,
    })

In [ ]:
feedbacks_df

In [ ]:
insights_df["related_feedback"] = [[] for _ in range(len(insights_df))]

for i, row in feedbacks_df.iterrows():
    for j in row["insights"]:
        insights_df["related_feedback"].iloc[int(j)] = row['_id'] #[int(i)]

insights_df["childrens"] = [[] for _ in range(len(insights_df))]

insights_df.head()

# Insights categorisation

### Tagging

In [ ]:


for i, filter in filters_df.iterrows():
    prompt_tags += '\n\n'+filter["Name"]#+' ('+filter["_id"] +')'
    tags = tags_df[tags_df["Filter"] == filter["_id"]]
    for _, tag in tags.iterrows():
        prompt_tags += '\n'+"- "+tag["Name"]+' ('+tag["_id"] +')'

print(prompt_tags)


In [ ]:
with open(prompts_path+'prompt_categorsiation.txt') as f:
    prompt_categorsiation = PromptTemplate.from_template(f.read())

In [ ]:
class FirstInsight(BaseModel):
    tags_id: List[str] = Field(description="Identifiants des tags de l'insight")
    content: str = "" #Field(description="Point intéressant a retenir du commentaire.")

    def __str__(self):
        return '- ' + self.content + "\nTypes: " + ', '.join(self.insight_types)

In [ ]:
categorsiation_parser = PydanticOutputParser(pydantic_object=FirstInsight)

prompt_categorsiation = PromptTemplate.from_template(
    template= prompt_template_categorsiation,
    partial_variables= {"format_instructions": categorsiation_parser.get_format_instructions()},
)

prompts = []
for insight in insights_df["content"]:
    context = deepcopy(feedback_context)
    context["insight"] = insight
    prompts.append(prompt_categorsiation.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, categorsiation_parser)


In [ ]:

insights_df["tag"] = [rep.tags_id for rep in parsed_responses]
#insights_df["Insights"] = [[] for rep in parsed_responses]


### Types affectation

In [ ]:
prompt_types = ""

for _, tag in types_df.iterrows():
    prompt_types += '\n'+"- "+tag["Title"]+' ('+tag["_id"] +') : ' + tag["Definition"]

print(prompt_types)

In [ ]:
categorsiation_parser = PydanticOutputParser(pydantic_object=FirstInsight)

prompt_categorsiation = PromptTemplate.from_template(
    template= prompt_template_types,
    partial_variables= {"format_instructions": categorsiation_parser.get_format_instructions()},
)

prompts = []
for insight in insights_df["content"]:
    context = deepcopy(feedback_context)
    context["insight"] = insight
    prompts.append(prompt_categorsiation.invoke(context))

#print(prompts[0].text)

In [ ]:
parsed_responses = safe_async_analysis(prompts, categorsiation_parser)


In [ ]:
insights_df["type"] = [rep.insight_type for rep in parsed_responses]

In [ ]:
feedbacks_df.to_csv(project_path+'/feedbacks.csv', index_label='Index')
insights_df.to_csv(project_path+'/insights.csv', index_label='Index')

# Data cleaning

# Visualisation

In [ ]:
insight_layers = [
    pd.DataFrame(bubble_client.get_objects(
        "python_insight",
        [
            BubbleField("step") == i+1,
            BubbleField("company") == company_id,
            ],
    )) for i in range(n_layers)
]

In [ ]:
insight_layers[0].tail()

In [ ]:
sentences = insight_layers[0]["content"]
sentence_embeddings = embedding_model.encode(sentences)
sentence_embeddings.shape

In [ ]:
insight_layers[0]['parent']

In [ ]:
insight_layers[0]

In [ ]:
def to_int(i):
    try:
        return int(i)
    except:
        return -1

for layer in insight_layers:
    layer['parent'] = layer['parent'].apply(to_int)


In [ ]:
list(insight_layers[1]["content"])

In [ ]:
for i, layer in enumerate(insight_layers):
    print(list(insight_layers[0][insight_layers[0]['parent'] == 'None']["content"]))

In [ ]:
sum(insight_layers[0]['parent']<0)

In [ ]:
insight_layers[1].iloc[insight_layers[0]['parent'], "content"]

In [ ]:
insight_layers[0].loc[0, "cluster"] == 0

In [ ]:
map_to_parent(0, insight_layers[1])

In [ ]:
insight_layers[1].loc[0, 'parent']

In [ ]:
#@Insight Plot the archive {display-mode: "form"}

# UMAP reduces the dimensions from 1024 to 2 dimensions that we can plot
reducer = umap.UMAP(n_neighbors=15)
umap_embeds = reducer.fit_transform(sentence_embeddings)

def map_to_parent(i, parents_df):
    try:
        return parents_df.loc[i, 'content']
    except:
        return ""
    
# Prepare the data to plot and interactive visualization
# using Altair
df_explore = pd.DataFrame(data={
    'content': insight_layers[0]['content'], 
    'parent': insight_layers[0]['parent'].apply(lambda x: map_to_parent(x, insight_layers[1])),
    'cluster': insight_layers[0]['cluster'].astype(str),
    })
df_explore['x'] = umap_embeds[:,0]
df_explore['y'] = umap_embeds[:,1]
df_explore


In [ ]:

# Plot
chart = alt.Chart(df_explore).mark_circle(size=60).encode(
    x=#'x',
    alt.X('x',
        scale=alt.Scale(zero=False)
    ),
    y=
    alt.Y('y',
        scale=alt.Scale(zero=False)
    ),
    color='cluster',
    tooltip=['content', "parent"]
).properties(
    width=700,
    height=400
)
chart.interactive()

TF-IDF

In [ ]:
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

nltk.download('punkt')

In [ ]:
def td_idf(documents)
    vectorizer = TfidfVectorizer()
    vectors = vectorizer.fit_transform(documents)
    feature_names = vectorizer.get_feature_names_out()
    dense = vectors.todense()
    denselist = dense.tolist()
    df = pd.DataFrame(denselist, columns=feature_names)
    df = df[df.columns.difference(stopwords.words('french'))]


In [ ]:
df = td_idf(feedbacks_df['content'])
#print('\n'.join(df.columns))

In [ ]:
#print('\n'.join(df.columns))

In [ ]:

def get_top_two_columns(row):
    top_two_indexes = row.nlargest(5).index.tolist()
    return top_two_indexes

top_two_columns_df = df.apply(get_top_two_columns, axis=1)

print(top_two_columns_df)

In [ ]:
#print('\n'.join(insights_df['content']))